In [18]:
import gensim
from gensim import corpora
from gensim.models import LsiModel

In [17]:
book_names = ["A dive into maths","Advanced maths","common probability distributions","the maths of probability", \
              "Was Thanos right?", "Stark being stark !", "A study of marvel characters: Stark vs Dr. strange"]
book_names_split = [[word.lower() for word in doc.split(" ")] for doc in book_names]
##You can use a very advanced tokenizer here from gensim/nltk ...
print(book_names_split)

[['a', 'dive', 'into', 'maths'], ['advanced', 'maths'], ['common', 'probability', 'distributions'], ['the', 'maths', 'of', 'probability'], ['was', 'thanos', 'right?'], ['stark', 'being', 'stark', '!'], ['a', 'study', 'of', 'marvel', 'characters:', 'stark', 'vs', 'dr.', 'strange']]


In [22]:
dictionary = corpora.dictionary.Dictionary(book_names_split)
print(dictionary.token2id)

{'a': 0, 'dive': 1, 'into': 2, 'maths': 3, 'advanced': 4, 'common': 5, 'distributions': 6, 'probability': 7, 'of': 8, 'the': 9, 'right?': 10, 'thanos': 11, 'was': 12, '!': 13, 'being': 14, 'stark': 15, 'characters:': 16, 'dr.': 17, 'marvel': 18, 'strange': 19, 'study': 20, 'vs': 21}


In [23]:
###Now let's create corpus
corpus = [dictionary.doc2bow(doc) for doc in book_names_split]

In [28]:
### Now let's fit our model
model = LsiModel(corpus=corpus, id2word=dictionary)

In [43]:
model.show_topics(2,10)

[(0,
  '-0.526*"stark" + -0.328*"of" + -0.327*"a" + -0.273*"vs" + -0.273*"characters:" + -0.273*"strange" + -0.273*"marvel" + -0.273*"study" + -0.273*"dr." + -0.126*"being"'),
 (1,
  '0.537*"maths" + -0.474*"stark" + 0.307*"probability" + 0.249*"of" + -0.248*"!" + -0.248*"being" + 0.227*"the" + 0.221*"a" + 0.198*"dive" + 0.198*"into"')]

## Now let's dive into the results

In [62]:
#given a new book, which topic does it fall into and what's the nearest match?
doc = "Elementary maths"
doc_preprocessed = [word.lower() for word in doc.split(" ")]
doc_bow = dictionary.doc2bow(doc_preprocessed)
per_topic_match = model[doc_bow]
print("The per topic match of document: {}".format(per_topic_match))
matched_topic = sorted( per_topic_match, key=lambda x:x[1],  reverse=True)[0][0]
print("Max match with the following topic : {}".format(model.show_topic(matched_topic)))
print(model.show_topic(2))

The per topic match of document: [(0, -0.12210648790137504), (1, 0.5365007933834613), (2, -0.40109536529899326), (3, 0.2500607809184919), (5, 0.22137366995885238), (6, -0.25371236704841915)]
Max match with the following topic : [('maths', 0.5365007933834608), ('stark', -0.4742372442955083), ('probability', 0.30733817246172546), ('of', 0.24935607968057377), ('!', -0.24836733040791195), ('being', -0.24836733040791195), ('the', 0.22685866316025813), ('a', 0.22080547761328784), ('dive', 0.19830806109297233), ('into', 0.19830806109297228)]
[('stark', -0.42220870194798404), ('maths', -0.4010953652989929), ('being', -0.31783160432184565), ('!', -0.31783160432184565), ('probability', -0.28568621125979543), ('study', 0.2134545066957073), ('strange', 0.2134545066957073), ('characters:', 0.2134545066957073), ('marvel', 0.21345450669570729), ('dr.', 0.21345450669570729)]


In [ ]:
## Now let's create an index 


In [ ]:
#Givena query, what's the nearest matches?